In [ ]:
import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
with open("/content/drive/MyDrive/Minor Project 2/data files4/word_to_idx.pkl","rb") as w2i:
    word_to_idx=pickle.load(w2i)
with open("/content/drive/MyDrive/Minor Project 2/data files4/idx_to_word.pkl","rb") as i2w:
    idx_to_word=pickle.load(i2w)
with open("/content/drive/MyDrive/Minor Project 2/data files4/embedding_matrix.pkl","rb") as em:
    embedding_matrix=pickle.load(em)
with open("/content/drive/MyDrive/Minor Project 2/data files4/x_train.pkl","rb") as tfi:
    x_train=pickle.load(tfi)
with open("/content/drive/MyDrive/Minor Project 2/data files4/y_train.pkl","rb") as tds:
    y_train=pickle.load(tds)

In [ ]:
x_t,x_v,y_t,y_v = train_test_split(x_train,y_train,test_size=0.1,random_state=0,stratify=y_train)

In [ ]:
max_len = 0
for cap in x_t:
    max_len = max(max_len,len(cap.split()))
print(max_len)

40


In [ ]:
def data_generator(x):
  x_array = []
  for desc in x:
    seq = [word_to_idx[word] for word in desc.split() if word in word_to_idx]
    xi = pad_sequences([seq],maxlen=max_len,value=0,padding='post')[0]
    x_array.append(xi)
  return x_array

In [ ]:
x_arr_t = data_generator(x_t)
x_arr_v = data_generator(x_v)

In [ ]:
x_arr_t = np.array(x_arr_t)
y_arr_t = np.array(y_t)
x_arr_v = np.array(x_arr_v)
y_arr_v = np.array(y_v)

In [ ]:
x_arr_v.shape

(13496, 40)

In [ ]:
y_arr_t = y_arr_t.reshape((y_arr_t.shape[0],1))
y_arr_t.shape

(121455, 1)

In [ ]:
y_arr_v = y_arr_v.reshape((y_arr_v.shape[0],1))
y_arr_v.shape

(13496, 1)

In [ ]:
np.unique(y_arr_t,return_counts=True)

(array([0, 1]), array([97735, 23720]))

In [ ]:
vocab_size = embedding_matrix.shape[0]
emb_dim = embedding_matrix.shape[1]
print(vocab_size,emb_dim)
print(max_len)

12849 300
40


In [ ]:
from keras.models import Model, load_model,Sequential
from keras.layers import Input,Dropout,Embedding,LSTM,Dense,Attention,Bidirectional, Layer
from tensorflow.keras import backend as K
from keras.callbacks import ModelCheckpoint

In [ ]:
class attention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [ ]:
model = Sequential()
model.add(Input(shape=(max_len,)))
model.add(Embedding(vocab_size,emb_dim, input_length=max_len))
model.add(Dropout(0.2))
model.add((LSTM(256, return_sequences=True,activation='relu')))
model.add(Dropout(0.2))
model.add((LSTM(128, return_sequences=True,activation='relu')))
model.add(Dropout(0.2))
model.add(attention(return_sequences=True)) # receive 3D and output 3D
model.add(LSTM(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 300)           3854700   
_________________________________________________________________
dropout (Dropout)            (None, 40, 300)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 40, 256)           570368    
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 128)           197120    
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
attention (attention)        (None, 40, 128)           1

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 300)           3854700   
_________________________________________________________________
dropout (Dropout)            (None, 40, 300)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 40, 256)           570368    
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 128)           197120    
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
attention (attention)        (None, 40, 128)           1

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
a = np.unique(y_arr_t, return_counts=True)
print(np.unique(y_arr_t, return_counts=True))
print(np.unique(y_arr_v, return_counts=True))
print(a[1][0]/a[1][1])

(array([0, 1]), array([97735, 23720]))
(array([0, 1]), array([10860,  2636]))
4.120362563237774


In [ ]:
class_wights={0:1,1:4.2}

In [ ]:
class_wights

{0: 1, 1: 4.2}

In [ ]:
y_arr_t.shape[0]/64

1897.734375

In [ ]:
filepath = "/content/drive/MyDrive/Minor Project 2/data files4/model1/new_model_{epoch:02d}"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [ ]:
model.fit(x=x_arr_t, y=y_arr_t, batch_size=64, epochs=20,validation_data=(x_arr_v,y_arr_v), shuffle=True,callbacks=[checkpoint], class_weight=class_wights, steps_per_epoch=1897, use_multiprocessing=True)

Epoch 1/20
1897/1897 [==============================] - 643s 336ms/step - loss: 1.1720 - accuracy: 0.4200 - val_loss: 0.2861 - val_accuracy: 0.9089
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Minor Project 2/data files4/model1/new_model_01/assets
Epoch 2/20
1897/1897 [==============================] - 628s 331ms/step - loss: 0.6096 - accuracy: 0.9236 - val_loss: 0.1461 - val_accuracy: 0.9442
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Minor Project 2/data files4/model1/new_model_02/assets
Epoch 3/20
1897/1897 [==============================] - 629s 332ms/step - loss: 0.2135 - accuracy: 0.9447 - val_loss: 0.1173 - val_accuracy: 0.9538
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Minor Project 2/data files4/model1/new_model_03/assets
Epoch 4/20
1897/1897 [==============================] - 616s 325ms/step - loss: 0.2590 - accuracy: 0.9416 - val_loss: 0.1642 - val_accuracy: 0.9408
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Minor Proje

KeyboardInterrupt: ignored

In [ ]:
model.save("/content/drive/MyDrive/Minor Project 2/data files/model_2",save_format='tf')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Minor Project 2/data files/model_2/assets
